In [23]:
import pickle
from multiprocessing.pool import Pool

import imdb
import numpy as np
import pandas as pd

with open("data/actors.pkl", "rb") as f:
    df = pd.DataFrame(sorted(pickle.load(f)), columns=["nconst"])

def get_country(nconst):
    ia = imdb.IMDb()
    person = ia.get_person(nconst[2:])
    if "birth info" in person.data:
        birth_place = person.data["birth info"]["birth place"]
        country = birth_place.split(",")[-1].strip()
        country = country.split("now")[-1].strip()
        country = "".join([ch for ch in country if ch.isalpha() or ch == " "])
    else:
        birth_place = person.data["birth info"]["birth place"]
        country = ""
    print(country + "|" + birth_place)
    return country

def get_countries(df):
    df["birth_country"] = df["nconst"].apply(get_country)
    return df

def parallelize_dataframe(df, func, n_cores=32):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [24]:
df_ = parallelize_dataframe(df, get_countries)

India|Kanigiri, Andhra Pradesh, India
USA|Omaha, Nebraska, USA
France|Alger, France
USA|The Bronx, New York City, New York, USA
France|Paris, France
Sweden|Stockholm, Sweden
USA|New York City, New York, USA


Process ForkPoolWorker-100:
Process ForkPoolWorker-99:
Process ForkPoolWorker-101:
Process ForkPoolWorker-102:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/usr/lib/python3.8/multiprocessing/pool.py",

KeyboardInterrupt: 

In [ ]:
df_.to_pickle("data/birth_countries.pkl")
df_